<span style='background:#FFF8E1; font-size:30px; display:block; border-left:40px solid #FFECB3; padding-left:40px;'>
<b>
    STEPWISE METEOSCREENING DB
</b>
</span>

<span style='background:#DCEDC8; font-size:30px; display:block; border-left:40px solid #C5E1A5; padding-left:40px;'>
<b>
    Example notebook for two air temperature variables from CH-DAV
</b>
</span>

**notebook version: `2.1`**

# **ABOUT THIS NOTEBOOK**

In this example notebook, the raw data for the variables `TA_NABEL_T1_35_1` and `TA_PRF_T1_35_1` (both are measurement `TA`)  
are downloaded from the database, quality-screened, resampled and then uploaded to the database using the [StepwiseMeteoScreeningDb](https://gitlab.ethz.ch/diive/diive/-/blob/main/diive/pkgs/qaqc/meteoscreening.py) class in [diive](https://gitlab.ethz.ch/diive/diive).  

**Here is an overview of what is done in this notebook**:
- (1) **USER SETTINGS**: First, we specify some general settings for the site and variable
- (2) **AUTO-SETTINGS**: With this info, some settings are set automatically
- (3) **DOWNLOAD DATA FROM DATABASE WITH `dbc-influxdb`**: Then original raw data are downloaded from the database. This step uses the Python module  
`dbc-influxdb` to download the data.
- (4) **METEOSCREENING WITH `diive`**: The downloaded data is then quality-screened using the Python library `diive`. The screening is done on the *high-resolution* data,  
i.e., in the original time resolution of the raw data. After the screening, data are resampled to 30MIN time resolution, the same resolution that we typically use for the fluxes.
- (5) **UPLOAD DATA TO THE DATABASE WITH `dbc-influxdb`**: The screened and resampled data are then uploaded to the database.

For more info about Stepwise MeteoScreeningDb, see notebook **Info - StepwiseMeteoScreeningDb**.

---

# **USER SETTINGS** (please adjust)

## Site

In [ ]:
SITE = 'ch-cha'
SITE_LAT = 47.210222
SITE_LON = 8.410444

## Variables to screen
- Specify variables as shown in the database
- `FIELDS`: Variables are called FIELDS in the database. InfluxDB stores variable names as '_field'. You can specify multiple fields, given as a list e.g. `['TA_NABEL_T1_35_1', 'TA_T1_20_1]`
- `MEASUREMENTS`: Only **one** measurement allowed. Measurement name that is used to group similar variables together, e.g., `TA` contains all air temperature variables, `SW` are all short-wave radiation measurements and `SWC` all soil water measurements. 

In [ ]:
FIELDS = ['TA_T1_2_1']
MEASUREMENT = 'TA'

## Time range to screen
- `START`: Screen data starting with this date (the start date itself **IS** included)
- `STOP`: Screen data before this date (the stop date itself **IS NOT** included)

In [ ]:
START = '2022-05-01 00:00:01'
STOP = '2022-05-15 00:00:01'

---

# **AUTO-SETTINGS**

## Data settings

In [ ]:
DATA_VERSION = 'raw'
TIMEZONE_OFFSET_TO_UTC_HOURS = 1  # Timezone, e.g. "1" is translated to timezone "UTC+01:00" (CET, winter time)
RESAMPLING_FREQ = '30T'  # During MeteoScreening the screened high-res data will be resampled to this frequency; '30T' = 30-minute time resolution
RESAMPLING_AGG = 'mean'  # The resampling of the high-res data will be done using this aggregation methos; e.g., 'mean'
DIRCONF = r'P:\Flux\RDS_calculations\_scripts\_configs\configs'  # Location of configuration files, needed e.g. for connection to database
# DIRCONF = r'F:\Sync\luhk_work\20 - CODING\22 - POET\configs'

## Imports

In [ ]:
from datetime import datetime
import pkg_resources
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import pandas as pd
from bokeh.plotting import output_notebook, show
output_notebook()
from dbc_influxdb import dbcInflux
from diive.pkgs.qaqc.meteoscreening import StepwiseMeteoScreeningDb
from diive.core.plotting.timeseries import TimeSeries
# version_diive = pkg_resources.get_distribution("diive").version
# print(f"diive version: v{version_diive}")
version_dbc_influxdb = pkg_resources.get_distribution("dbc_influxdb").version
print(f"dbc-influxdb version: v{version_dbc_influxdb}")

## Automatic settings (do not adjust):

In [ ]:
BUCKET_RAW=f'{SITE}_raw'  # The 'bucket' where data are stored in the database, e.g., 'ch-lae_raw' contains all raw data for CH-LAE
BUCKET_PROCESSING=f'{SITE}_processing'  # The 'bucket' where data are stored in the database, e.g., 'ch-lae_processing' contains all processed data for CH-LAE
print(f"Bucket containing raw data (source bucket): {BUCKET_RAW}")
print(f"Bucket containing processed data (destination bucket): {BUCKET_PROCESSING}")

## Connect to database

In [ ]:
dbc = dbcInflux(dirconf=DIRCONF)

## Check available variables
- Since we are now connected to the database, we can make a quick check if the variable(s) we want to screen are really in the database.  
Below you can show show all available fields in the selected measurement.
- However, if the desired variable(s) are indeed listed here, it does not necessarily mean that they are also available during the  
selected **time period**. This can be the case if the variable(s) for that time period were not uploaded to the database.

In [ ]:
display(dbc.show_fields_in_measurement(bucket=BUCKET_RAW, measurement=MEASUREMENT))

---

# **DOWNLOAD DATA FROM DATABASE WITH `dbc-influxdb`**
- Here the Python library `dbc-influxdb` is used to download data from the database.
- The name of the library means **d**ata**b**ase **c**ommunication using **influxdb**.
- You can find the source code of `dbc-influxdb` [here](https://gitlab.ethz.ch/poet/dbc-influxdb).
- There is no need to adjust the settings here, the variables given earlier (above) are directly used.
- At this point we have already established the connection to the database and stored the connection details in the variable `dbc`.
- Now we can use the method `dbc.download()` to start the data download.

## Download data

In [ ]:
%%time
data_simple, data_detailed, assigned_measurements = dbc.download(
    bucket=BUCKET_RAW,
    measurements=[MEASUREMENT],
    fields=FIELDS,
    start=START,
    stop=STOP,
    timezone_offset_to_utc_hours=TIMEZONE_OFFSET_TO_UTC_HOURS,
    data_version=DATA_VERSION
)

## Check downloaded data
### `dbc` downloads three variables:
- `data_simple`: This is the simplest data download. It contains only the high-resolution variable(s) from the specified measurement. Each variable is in a separate column. Since different variables, can have different time resolutions, the highest time resolution across the variables is used as the index. This means that lower resolution variables will show gaps in the higher resolution timestamp.
- `data_detailed`: This is the most important variable for the MeteoScreening from the database, because it contains not only the high-resolution time series of the variable(s), but also their tags. The tags are important when uploading data to the database. This is a very special format, because data for each variable are stored in a dictionary. A dictionary is a data structure that stores key-value pairs. The key is the variable name (e.g., `TA_NABEL_T1_35_1`) and the value is a complete dataframe that contains the time series of the respective variable and all tags.
- `assigned_measurements`: An auxiliary variable that is useful to check whether the measurement of the variable(s) is correct. In case we are screening air temperatures, the measurement must be `TA`.

#### `data_simple`
- Simply contains the time series of the measured variable(s) in a nice-to-look-at dataframe.

In [ ]:
data_simple

#### `data_detailed`
- Dictionary of key-value pairs: the variable name is the key, the respective data (measured time series and tags) are the values

In [ ]:
data_detailed

<br>
Since the output above is not so nice to look at, we loop through the dictionary and display the first and last records for each variable:

In [ ]:
for varname, frame in data_detailed.items():
    print(f"{'-' * 40}\nData for {varname}:")
    display(frame)

#### `assigned_measurements`
- Contains the *automatically* detected measurement for the selected variables, useful as check to make sure the variable was properly identified.

In [ ]:
assigned_measurements

<br>
It is possible that a variable that we want to screen is not available in the database (oh no). To find out which variable that might be we have this piece of code:

In [ ]:
print(f"Data available for: {data_detailed.keys()}\n")
vars_not_available = [v for v in FIELDS if v not in data_detailed.keys()]
print(f"No data available for the following variables: {vars_not_available}")

## Save downloaded high-res raw data to file (optional)
- If you really want, you can download the full resolution data to a file. This can be useful if you want to do the screening elsewhere.

In [ ]:
# data_detailed['TA_NABEL_T1_35_1'].to_csv("mylovelyhorse.csv")

## Plot downloaded high-res data
- Yes, this is indeed a plot of the downloaded full-resolution data.
- Because we have plenty of time and space, each variable gets it's own plot.
- If there is only one variable, then there is only one plot.

In [ ]:
for varname, frame in data_detailed.items():
    TimeSeries(series=frame[varname]).plot_interactive()
    # TimeSeries(series=frame[varname]).plot()

---

# **METEOSCREENING WITH `diive`**

## Start new MeteoScreening session

In [ ]:
mscr = StepwiseMeteoScreeningDb(site=SITE,
                                data_detailed=data_detailed,
                                measurement=MEASUREMENT,
                                fields=FIELDS,
                                site_lat=SITE_LAT,
                                site_lon=SITE_LON)

<br>
Now let's take a look at the data that will be screened:

In [ ]:
mscr.showplot_orig(interactive=True)

## **QC FLAGS**: generate quality flags on high-resolution data (if needed)

<span style='background:#FFEBEE; font-size:16px; line-height: 100px; display:block; border-left:40px solid #FFCDD2; padding-left:40px;'>
<p>
    If the plots look good and you think the data do not require any screening, you can skip this section and jump directly to <b>Corrections</b> or <b>Resampling</b>.
</p>
</span>

### **Flag: Missing values test** (always needed)

Must be used for: **-ALL-**  
*Generate flag that indicates missing records in data*

In [ ]:
mscr.flag_missingvals_test()

In [ ]:
mscr.addflag()

### **Flag: Outlier detection, z-score over all data** (*flag_outliers_zscore_test*)

Can be used for: **--ALL--**  
This is a general purpose test that can already solve many outlier issues.  
*Identify outliers based on the z-score of records*

In [ ]:
 mscr.flag_outliers_zscore_test(threshold=2.5, showplot=True, verbose=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, z-score over all data with IQR** (*flag_outliers_zscoreiqr_test*)

Can be used for: **--ALL--**  
*Identify outliers based on max z-scores in the interquartile range data*

In [ ]:
# mscr.flag_outliers_zscoreiqr_test(factor=3, showplot=True, verbose=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, z-score over all data, separate for daytime and nighttime** (*flag_outliers_zscore_dtnt_test*)

Can be used for: **TA**  
*Identify outliers based on the z-score, separately for daytime and nighttime*

In [ ]:
# mscr.flag_outliers_zscore_dtnt_test(threshold=2.5, showplot=True, verbose=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, Seasonal trend decomposition (residuals, IQR, z-score)** (*flag_outliers_stl_riqrz_test*)

Can be used for: **TA**  
*Identify outliers based on seasonal-trend decomposition and z-score calculations*

In [ ]:
mscr.flag_outliers_stl_riqrz_test(zfactor=2, decompose_downsampling_freq='2H', showplot=True, repeat=False)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, Increments z-score** (*flag_outliers_increments_zcore_test*)

Can be used for: **-ALL-**  
*Identify outliers based on the z-score of record increments*

In [ ]:
mscr.flag_outliers_increments_zcore_test(threshold=85, showplot=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, Absolute limits** (*flag_outliers_abslim_test*)

Can be used for: **-ALL-**  
*Generate flag that indicates if values in data are outside the specified range*

In [ ]:
MIN = -20
MAX = 50
mscr.flag_outliers_abslim_test(min=MIN, max=MAX, showplot=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, Local SD** (*flag_outliers_localsd_test*)

Can be used for: **-ALL-**  
*Identify outliers based on the local standard deviation*

In [ ]:
N_SD = 1
mscr.flag_outliers_localsd_test(n_sd=N_SD, showplot=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, Thymeboost** (*flag_outliers_thymeboost_test*)

Can be used for: **TA**  
*Identify outliers based on [thymeboost](https://github.com/tblume1992/ThymeBoost)*

In [ ]:
mscr.flag_outliers_thymeboost_test(showplot=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, manual removal** (*flag_manualremoval_test*)

Can be used for: **-ALL-**  
*Generate flag for data points that should be removed*

In [ ]:
mscr.showplot_cleaned(interactive=True)

In [ ]:
REMOVE_DATES = [
    '2022-05-12 17:40:30',
    # ['2021-08-05', '2021-08-19']
]
mscr.flag_manualremoval_test(remove_dates=REMOVE_DATES, showplot=True, verbose=True)

In [ ]:
mscr.addflag()

### **Flag: Outlier detection, local outlier factor** (*flag_outliers_lof_test*)

Can be used for: **-ALL-**  
*Identify outliers based on the local outlier factor*

In [ ]:
mscr.flag_outliers_lof_test(n_neighbors=None, contamination='auto', showplot=True, verbose=True)

In [ ]:
# mscr.addflag()

### **Flag: Outlier detection, local outlier factor daytime/nighttime** (*flag_outliers_lof_dtnt_test*)

Can be used for: **-ALL-**  
*Identify outliers based on the local outlier factor, done separately for daytime and nighttime data*

In [ ]:
mscr.flag_outliers_lof_dtnt_test(n_neighbors=None, contamination='auto', showplot=True, verbose=True)

In [ ]:
# mscr.addflag()

### **QCF**: calculate overall quality flag for high-resolution data

In [ ]:
mscr.calc_qcf()

#### Reports

In [ ]:
mscr.report_qcf_evolution()

In [ ]:
mscr.report_qcf_series()

In [ ]:
mscr.report_qcf_flags()

#### Plots

In [ ]:
mscr.showplot_qcf_heatmaps(figsize=(18,8))

In [ ]:
mscr.showplot_qcf_timeseries(figsize=(18,8))

## **CORRECTIONS**: done on high-resolution data (if needed)

<span style='background:#FFEBEE; font-size:16px; line-height: 100px; display:block; border-left:40px solid #FFCDD2; padding-left:40px;'>
<p>
    Currently only needed for <b>SW, PPFD, RH</b>.
</p>
</span>

### **Correction: Remove radiation zero offset** (*correction_remove_radiation_zero_offset*)
Can be used for: **SW_IN, SW_OUT, PPFD_IN, PPFD_OUT**  
*Remove nighttime offset from all radiation data and set nighttime to zero*

In [ ]:
# mscr.correction_remove_radiation_zero_offset()

### **Correction: Remove relative humidity offset** (*correction_remove_relativehumidity_offset*)
Can be used for: **RH**  
*Remove relative humidity offset, works for relative humidity data where maximum values should not exceed 100%*  

In [ ]:
# mscr.correction_remove_relativehumidity_offset()

### **Correction: Set to max threshold** (*correction_setto_max_threshold*)
Can be used for: **--ALL--**  
*Set values above threshold to threshold*

In [ ]:
# mscr.correction_setto_max_threshold(threshold=9999)

### **Correction: Set to min threshold** (*correction_setto_min_threshold*)
Can be used for: **--ALL--**  
*Set values below threshold to threshold*

In [ ]:
# mscr.correction_setto_min_threshold(threshold=9999)

## **RESAMPLING** <a class="anchor" id="resampling"></a>

In [ ]:
mscr.resample(to_freqstr='30T', agg='mean', mincounts_perc=.25)

In [ ]:
mscr.showplot_resampled()

---

# **UPLOAD DATA TO THE DATABASE WITH `dbc-influxdb`**

In [ ]:
print(f"Data will be uploaded to bucket {BUCKET_PROCESSING}")

In [ ]:
for v in mscr.resampled_detailed.keys(): 
    m = assigned_measurements[v]    
    dbc.upload_singlevar(to_bucket=BUCKET_PROCESSING,
                         to_measurement=m,
                         var_df=mscr.resampled_detailed[v],
                         timezone_of_timestamp='UTC+01:00')
    # dbc.upload_singlevar(to_bucket='test', to_measurement=m, var_df=vars_qc_resampled[v], timezone_of_timestamp='UTC+01:00')

---

# **DOWNLOAD FROM DATABASE**

In [ ]:
dbc = dbcInflux(dirconf=DIRCONF)
dbc.show_fields_in_measurement(bucket=BUCKET_PROCESSING, measurement='TA')

In [ ]:
data_simple, data_detailed, assigned_measurements = dbc.download(bucket=BUCKET_PROCESSING,
                                                                 measurements=[MEASUREMENT],
                                                                 fields=FIELDS,
                                                                 start=START,
                                                                 stop=STOP,
                                                                 timezone_offset_to_utc_hours=TIMEZONE_OFFSET_TO_UTC_HOURS,
                                                                 data_version='meteoscreening')

In [ ]:
data_simple

---

# **FINISH**
Congratulations, you reached the end of this notebook! Before you go let's store your finish time.

In [ ]:
dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Finished. {dt_string}")